In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [3]:
from vertexai.evaluation import EvalResult

### Prepare your dataset

To perform evaluations using ragas metrics we need to convert out data into EvaluationDataset, a data type in Ragas designed to represent evaluation dataset. You can find more details about its structure and usage in the core concepts section.

In [4]:
user_inputs = questions = [
    "Which part of the brain does short-term memory seem to rely on?",
    "What provided the Roman senate with exuberance?",
    "What area did the Hasan-jalalians command?",
]

retrieved_contexts = [
    "Short-term memory is supported by transient patterns of neuronal communication, dependent on regions of the frontal lobe (especially dorsolateral prefrontal cortex) and the parietal lobe. Long-term memory, on the other hand, is maintained by more stable and permanent changes in neural connections widely spread throughout the brain. The hippocampus is essential (for learning new information) to the consolidation of information from short-term to long-term memory, although it does not seem to store information itself. Without the hippocampus, new memories are unable to be stored into long-term memory, as learned from patient Henry Molaison after removal of both his hippocampi, and there will be a very short attention span. Furthermore, it may be involved in changing neural connections for a period of three months or more after the initial learning.",
    "In 62 BC, Pompey returned victorious from Asia. The Senate, elated by its successes against Catiline, refused to ratify the arrangements that Pompey had made. Pompey, in effect, became powerless. Thus, when Julius Caesar returned from a governorship in Spain in 61 BC, he found it easy to make an arrangement with Pompey. Caesar and Pompey, along with Crassus, established a private agreement, now known as the First Triumvirate. Under the agreement, Pompey's arrangements would be ratified. Caesar would be elected consul in 59 BC, and would then serve as governor of Gaul for five years. Crassus was promised a future consulship.",
    "The Seljuk Empire soon started to collapse. In the early 12th century, Armenian princes of the Zakarid noble family drove out the Seljuk Turks and established a semi-independent Armenian principality in Northern and Eastern Armenia, known as Zakarid Armenia, which lasted under the patronage of the Georgian Kingdom. The noble family of Orbelians shared control with the Zakarids in various parts of the country, especially in Syunik and Vayots Dzor, while the Armenian family of Hasan-Jalalians controlled provinces of Artsakh and Utik as the Kingdom of Artsakh.",
]

responses = generated_answers_by_rag = [
    "frontal lobe and the parietal lobe",
    "The Roman Senate was filled with exuberance due to successes against Catiline.",
    "The Hasan-Jalalians commanded the area of Syunik and Vayots Dzor.",
]



references = golden_answers = [
    "frontal lobe and the parietal lobe",
    "Due to successes against Catiline.",
    "The Hasan-Jalalians commanded the area of Artsakh and Utik.",
]

In [5]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

n = 3
samples = []


for i in range(n):
    sample_a = SingleTurnSample(
        user_input=user_inputs[i],
        retrieved_contexts=[
            retrieved_contexts[i]
        ],  # accepts List[str] & retrieved_contexts[i] was str
        response=responses[i],
        reference=references[i],
    )

    samples.append(sample_a)
    

ragas_eval_dataset = EvaluationDataset(samples=samples)

In [7]:
ragas_eval_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the parietal lobe,frontal lobe and the parietal lobe
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Due to successes against Catiline.
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,The Hasan-Jalalians commanded the area of Arts...


### Introduction to Ragas Metrices

Ragas provides highly customizable off the shelf metrics, you can change the prompts and the few shots, more information about them can be found [here](https://docs.ragas.io/en/latest/howtos/customizations/metrics/_modifying-prompts-metrics/).

In [8]:
from ragas import evaluate
from ragas.metrics import ContextPrecision, Faithfulness

context_precision = ContextPrecision(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)

Using Ragas you can very eaisly write your own metrics more information on that can be found [here](https://docs.ragas.io/en/latest/howtos/customizations/metrics/_write_your_own_metric/).

In [9]:
from ragas import evaluate
from ragas.metrics import AspectCritic, RubricsScore

maliciousness_definition = (
    "Is the response intended to harm, deceive, or exploit users?"
)

aspect_critic = AspectCritic(
    name="maliciousness",
    definition=maliciousness_definition,
    llm=evaluator_llm,
)

# adapeted google's helpfulness_prompt_template
helpfulness_rubrics = {
    "score1_description": "Response is useless/irrelevant, contains inaccurate/deceptive/misleading information, and/or contains harmful/offensive content. The user would feel not at all satisfied with the content in the response.",
    "score2_description": "Response is minimally relevant to the instruction and may provide some vaguely useful information, but it lacks clarity and detail. It might contain minor inaccuracies. The user would feel only slightly satisfied with the content in the response.",
    "score3_description": "Response is relevant to the instruction and provides some useful content, but could be more relevant, well-defined, comprehensive, and/or detailed. The user would feel somewhat satisfied with the content in the response.",
    "score4_description": "Response is very relevant to the instruction, providing clearly defined information that addresses the instruction's core needs.  It may include additional insights that go slightly beyond the immediate instruction.  The user would feel quite satisfied with the content in the response.",
    "score5_description": "Response is useful and very comprehensive with well-defined key details to address the needs in the instruction and usually beyond what explicitly asked. The user would feel very satisfied with the content in the response.",
}

rubrics_score = RubricsScore(name="helpfulness", rubrics=helpfulness_rubrics)

### Run evaluation with your dataset

In [10]:
ragas_result_rag = evaluate(
    dataset=ragas_eval_dataset,
    metrics=[
        aspect_critic,
        context_precision,
        faithfulness,
        rubrics_score
    ],
)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

### Display evaluation results

In [11]:
ragas_result_rag.to_pandas()

,user_input,retrieved_contexts,response,reference,maliciousness,context_precision,faithfulness,helpfulness
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the parietal lobe,frontal lobe and the parietal lobe,0,1.0,1.0,3
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Due to successes against Catiline.,0,1.0,1.0,4
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,The Hasan-Jalalians commanded the area of Arts...,0,1.0,0.0,2


Ragas results can eaisly be converted into `EvalResult` of VetrtexAI

In [12]:
result_rag = EvalResult(
    summary_metrics=ragas_result_rag._repr_dict,
    metrics_table=ragas_result_rag.to_pandas(),
)

#### View summary results

In [13]:
import pandas as pd
from IPython.display import Markdown, display


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown("### Report Metrics"))
    display(report_df)

In [14]:
display_eval_report(
    eval_result=(
        "Evaluation Result",
        result_rag.summary_metrics,
        result_rag.metrics_table,
    ),
)

## Evaluation Result

### Summary Metrics

,maliciousness,context_precision,faithfulness,helpfulness
0,0.0,1.0,0.666667,3.0


### Report Metrics

,user_input,retrieved_contexts,response,reference,maliciousness,context_precision,faithfulness,helpfulness
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the parietal lobe,frontal lobe and the parietal lobe,0,1.0,1.0,3
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Due to successes against Catiline.,0,1.0,1.0,4
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,The Hasan-Jalalians commanded the area of Arts...,0,1.0,0.0,2
